### İşlemlerin GPU üzerinde gerçekleştirilmesi

In [1]:
#GPU kullanmak için kimlik doğrulama işlemi
from google.colab import drive
drive.mount('/content/drive/')
#%cd /gdrive

Mounted at /content/drive/


###Google Drive gerekli konuma ulaşma

In [2]:
!ls "/content/drive/My Drive/"

BertTest  BertTest_2  Mustafa_Ozkan.docx  Python  Udemy_DerinOgrenmeyeGiris


In [3]:
import os
os.chdir("/content/drive/My Drive/BertTest/")

###Kullanacağımız donanımın özellikleri


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17078481233743816908, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5256190650527728021
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

### Veriyi İnceleme

In [4]:
#Gerekli kütüphanelerin kurulması
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
#Verinin okunması 
import pandas as pd
df = pd.read_csv('NamesAndAbstracts_With_Categories.csv')


In [6]:
#Toplam veri sayısını hesaplama
df.name_with_abstract.count()

1560

In [7]:
#Veriye ilişkin istatistikleri görme
df.category.value_counts()

Yapay Zeka                       598
Görüntü İşleme                   438
Haberleşme ve Sinyalleşme        217
IoT                              183
Artırılmış ve Sanal Gerçeklik    124
Name: category, dtype: int64

### Varsa Verideki Null Kategorileri Bulma

In [ ]:
ds = df[df.category.isnull()]

In [ ]:
print(ds)

Empty DataFrame
Columns: [name_with_abstract, category]
Index: []


### PRE Processing

In [8]:
#Gerekli kütüphanelerin kurulması
import pandas as pd
import numpy as np
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [9]:
#Veri setindeki ilk 10 değeri görelim
pp = df.loc[:, ['name_with_abstract', 'category']]
pp.head(10)

,name_with_abstract,category
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik
5,ÇEŞİTLİ ARAÇLARA UYGULANMAK ÜZERE SANAL GERÇEKLİK SİSTEMİ Buluş başta yolcu taşıma uçaklarında u...,Artırılmış ve Sanal Gerçeklik
6,BİR AKILLI CİHAZ UYGULAMASI Buluş akıllı cihazlarda (1) kullanılan uygulamalar ile ilgili olup ö...,Artırılmış ve Sanal Gerçeklik
7,KARMA GERÇEKLİK HAYALET PROTOKOLÜ Bu buluş deneyimcinin ve mekanın konumlandırma bilgileri ile e...,Artırılmış ve Sanal Gerçeklik
8,Doğum Kolaylaştırma Sistemi Doğum sürecindeki ve doğumdaki kadınların görme işitme ve koku alma ...,Artırılmış ve Sanal Gerçeklik
9,SANAL GERÇEKLİK ORTAMINDA OYUN TERAPİ SİSTEMİ Buluş dikkat konsantrasyon hafıza denge-koordinasy...,Artırılmış ve Sanal Gerçeklik


In [10]:
#Verisetimizin Pre-Processing aşamasından önce en çok kullanılan kelimelerini bulalım
df3 = (df['name_with_abstract'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('vals')
              .reset_index(name='count'))
print(df3)

                  vals  count
0                   ve   9624
1                  bir   6637
2                  ile   3701
3                 için   3539
4                   Bu   2903
...                ...    ...
39448  operatörleriyle      1
39449          olarak)      1
39450          %52'lik      1
39451        omurgasız      1
39452         uyararak      1

[39453 rows x 2 columns]


In [ ]:
#NLTK kütüphanesinden "Punkt Sentence Tokenizer" kullanarak bütün verimizi tokenlarına ayıralım
nltk.download('punkt')
pp['tokenized'] = pp['name_with_abstract'].apply(word_tokenize)
pp.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,name_with_abstract,category,tokenized
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik,"[NESNELERİ, ÜÇ, BOYUTLU, OLARAK, TARAYAN, CİHAZ, VE, ÖN, İZLEME, UYGULAMASI, Bu, buluş, mobil, v..."
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik,"[BİR, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜNDE, UYGULAMAK, İÇİN, BİR, EĞİTİM, SİSTEMİ, Buluş, bir, operatörü..."
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik,"[BİR, SANAL, GERÇEKLİĞİ, ARTTIRILMIŞ, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜ, Buluş, bir, operatörünün, eğiti..."
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik,"[Artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya..."
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik,"[GAYRİMENKULLERİ, SANAL, GERÇEKLİK, VASITASI, İLE, DENEYİMLEMEYİ, SAĞLAYAN, WEBVR, ALTYAPISINA, ..."


In [ ]:
#Verisetimizi tamamen küçük harfe çevirelim
pp['lower'] = pp['tokenized'].apply(lambda x: [word.lower() for word in x])
pp.head()

,name_with_abstract,category,tokenized,lower
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik,"[NESNELERİ, ÜÇ, BOYUTLU, OLARAK, TARAYAN, CİHAZ, VE, ÖN, İZLEME, UYGULAMASI, Bu, buluş, mobil, v...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil..."
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik,"[BİR, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜNDE, UYGULAMAK, İÇİN, BİR, EĞİTİM, SİSTEMİ, Buluş, bir, operatörü...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,..."
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik,"[BİR, SANAL, GERÇEKLİĞİ, ARTTIRILMIŞ, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜ, Buluş, bir, operatörünün, eğiti...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ..."
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik,"[Artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya..."
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik,"[GAYRİMENKULLERİ, SANAL, GERÇEKLİK, VASITASI, İLE, DENEYİMLEMEYİ, SAĞLAYAN, WEBVR, ALTYAPISINA, ...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi..."


In [ ]:
#String kütüphanesinden "punctuation" kullanarak versitemizdeki bütün noktalama işaretlerini kaldıralım
#Örnek punctuation outputu: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
punc = string.punctuation
pp['no_punc'] = pp['lower'].apply(lambda x: [word for word in x if word not in punc])
pp.head()

,name_with_abstract,category,tokenized,lower,no_punc
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik,"[NESNELERİ, ÜÇ, BOYUTLU, OLARAK, TARAYAN, CİHAZ, VE, ÖN, İZLEME, UYGULAMASI, Bu, buluş, mobil, v...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil..."
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik,"[BİR, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜNDE, UYGULAMAK, İÇİN, BİR, EĞİTİM, SİSTEMİ, Buluş, bir, operatörü...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,..."
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik,"[BİR, SANAL, GERÇEKLİĞİ, ARTTIRILMIŞ, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜ, Buluş, bir, operatörünün, eğiti...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ..."
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik,"[Artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya..."
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik,"[GAYRİMENKULLERİ, SANAL, GERÇEKLİK, VASITASI, İLE, DENEYİMLEMEYİ, SAĞLAYAN, WEBVR, ALTYAPISINA, ...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi..."


In [ ]:
#NLTK kütüphanesinden "stopwords" packageını kullanarak Türkçe dilindeki stopwordsleri verisetimizden kaldıralım
#Örnek Türkçe stopwords outputu: 'nerde', 'biz', 'bazı', 'siz', 'yani', 'ama', 'da', 'hep', 'niye', 'birkaç', 'mı', 'çok', 'her', 'için', 'ile', 'diye', 
# 'şey', 'bu', 'şu', 'de', 'belki', 'hepsi', 'nerede', 'ya', 'niçin', 'veya', 'o', 'nasıl', 'ne', 'daha', 'neden', 'kez', 'hem', 'az', 'defa', 'eğer', 'mu', 
# 'kim', 'çünkü', 'tüm', 'mü', 'acaba', 'nereye', 'aslında', 'sanki', 've', 'birşey', 'ki', 'en', 'biri', 'ise', 'gibi', 'hiç'

nltk.download('stopwords')
stop_words = set(stopwords.words('turkish'))
pp['stopwords_removed'] = pp['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
pp.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,name_with_abstract,category,tokenized,lower,no_punc,stopwords_removed
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik,"[NESNELERİ, ÜÇ, BOYUTLU, OLARAK, TARAYAN, CİHAZ, VE, ÖN, İZLEME, UYGULAMASI, Bu, buluş, mobil, v...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t..."
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik,"[BİR, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜNDE, UYGULAMAK, İÇİN, BİR, EĞİTİM, SİSTEMİ, Buluş, bir, operatörü...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,..."
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik,"[BİR, SANAL, GERÇEKLİĞİ, ARTTIRILMIŞ, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜ, Buluş, bir, operatörünün, eğiti...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ..."
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik,"[Artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a..."
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik,"[GAYRİMENKULLERİ, SANAL, GERÇEKLİK, VASITASI, İLE, DENEYİMLEMEYİ, SAĞLAYAN, WEBVR, ALTYAPISINA, ...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi..."


In [ ]:
#Regex kütüphanesini kullanarak verisetimizdeki bütün sayıları kaldıralım
import re
pp['digits_removed'] = pp['stopwords_removed'].apply(lambda x: [word for word in x if word not in re.findall('[0-9]+', word)])
pp.head()


,name_with_abstract,category,tokenized,lower,no_punc,stopwords_removed,digits_removed
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik,"[NESNELERİ, ÜÇ, BOYUTLU, OLARAK, TARAYAN, CİHAZ, VE, ÖN, İZLEME, UYGULAMASI, Bu, buluş, mobil, v...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t..."
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik,"[BİR, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜNDE, UYGULAMAK, İÇİN, BİR, EĞİTİM, SİSTEMİ, Buluş, bir, operatörü...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,..."
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik,"[BİR, SANAL, GERÇEKLİĞİ, ARTTIRILMIŞ, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜ, Buluş, bir, operatörünün, eğiti...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ..."
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik,"[Artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a..."
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik,"[GAYRİMENKULLERİ, SANAL, GERÇEKLİK, VASITASI, İLE, DENEYİMLEMEYİ, SAĞLAYAN, WEBVR, ALTYAPISINA, ...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi..."


In [ ]:
  #Snowballstemmer modülündeki TurkishStemmer kütüphanesini kullanarak verisetimize Türkçe stemming uygulayalım
  from snowballstemmer import TurkishStemmer
  turkStem=TurkishStemmer()
  pp['turkish_stem'] = pp['digits_removed'].apply(lambda x: [turkStem.stemWord(word) for word in x])
  pp.head()


,name_with_abstract,category,tokenized,lower,no_punc,stopwords_removed,digits_removed,turkish_stem
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik,"[NESNELERİ, ÜÇ, BOYUTLU, OLARAK, TARAYAN, CİHAZ, VE, ÖN, İZLEME, UYGULAMASI, Bu, buluş, mobil, v...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t...","[nesneleri̇, üç, boyutlu, olarak, taraya, ci̇haz, ön, i̇zle, uygulamasi, buluş, mobil, web, taba..."
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik,"[BİR, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜNDE, UYGULAMAK, İÇİN, BİR, EĞİTİM, SİSTEMİ, Buluş, bir, operatörü...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprül, vi̇nç, si̇mülatör, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir, oper..."
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik,"[BİR, SANAL, GERÇEKLİĞİ, ARTTIRILMIŞ, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜ, Buluş, bir, operatörünün, eğiti...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttiril, köprül, vi̇nç, si̇mülatör, buluş, bir, operatör, eğitilmes..."
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik,"[Artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a...","[artırıl, gerçeklik, sanal, gerçeklik, robotik, uygulama, müdahal, etme, sağlaya, akıllı, eldive..."
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik,"[GAYRİMENKULLERİ, SANAL, GERÇEKLİK, VASITASI, İLE, DENEYİMLEMEYİ, SAĞLAYAN, WEBVR, ALTYAPISINA, ...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitas, i̇, deneyi̇mlemeyi̇, sağlaya, webvr, altyapi, sa..."


In [ ]:
#Verisetimizden örnek görme
pp.values[29]

array(['BİR EĞİTİM VE ÖĞRETİM UYGULAMA YÖNTEMİ Eğitim ve öğretim sektöründe özellikle iş güvenliği ve işçi sağlığı eğitimlerinde katılımcılara eğitimlerini interaftik bir sanal ortamda vererek hem katılımı hem ilgiyi arttırırken dikkat dağılımını da ortadan kaldıran bir eğitim ve öğretim uygulama yöntemi ile ilgili olan bu buluş genel olarak sadece iş güvenliği ve işçi sağlığı eğitimlerinin verilmesi için eğitimci(3) ve katılımcıların(4) bir araya getirildiği ve eğitimin gerçekleştirildiği sınıf oda eğitim ortamı(5) eğitim konularının ve uygulamalarının tüm görsel ve dijital verilerinin barındırıldığı ve istenildiğinde eğitimci(3) kontrolünde verilerin gönderildiği ve yayınlandığı ana istemci(1) ana istemciden(1) gelen tüm görsel ve dijital verilerinin oynatıldığı ve katılımcı(4) eğitimcilere(3) sanal ortamda anı yaşatarak gösterimini sağlayan sanal gerçeklik modülü(2) katılımcının(4)eğitim konuları ve uygulamalarına sanal gerçeklik modülü(2) ile bağlantısıyla beraber uygulamada bireys

In [ ]:
#Verisetimizi arrayden çıkaralım
pp['final'] = pp['turkish_stem'].apply(', '.join)
pp.head()

,name_with_abstract,category,tokenized,lower,no_punc,stopwords_removed,digits_removed,turkish_stem,final
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik,"[NESNELERİ, ÜÇ, BOYUTLU, OLARAK, TARAYAN, CİHAZ, VE, ÖN, İZLEME, UYGULAMASI, Bu, buluş, mobil, v...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t...","[nesneleri̇, üç, boyutlu, olarak, taraya, ci̇haz, ön, i̇zle, uygulamasi, buluş, mobil, web, taba...","nesneleri̇, üç, boyutlu, olarak, taraya, ci̇haz, ön, i̇zle, uygulamasi, buluş, mobil, web, taban..."
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik,"[BİR, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜNDE, UYGULAMAK, İÇİN, BİR, EĞİTİM, SİSTEMİ, Buluş, bir, operatörü...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprül, vi̇nç, si̇mülatör, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir, oper...","bi̇r, köprül, vi̇nç, si̇mülatör, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir, opera..."
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik,"[BİR, SANAL, GERÇEKLİĞİ, ARTTIRILMIŞ, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜ, Buluş, bir, operatörünün, eğiti...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttiril, köprül, vi̇nç, si̇mülatör, buluş, bir, operatör, eğitilmes...","bi̇r, sanal, gerçekli̇ği̇, arttiril, köprül, vi̇nç, si̇mülatör, buluş, bir, operatör, eğitilmes,..."
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik,"[Artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a...","[artırıl, gerçeklik, sanal, gerçeklik, robotik, uygulama, müdahal, etme, sağlaya, akıllı, eldive...","artırıl, gerçeklik, sanal, gerçeklik, robotik, uygulama, müdahal, etme, sağlaya, akıllı, eldive,..."
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik,"[GAYRİMENKULLERİ, SANAL, GERÇEKLİK, VASITASI, İLE, DENEYİMLEMEYİ, SAĞLAYAN, WEBVR, ALTYAPISINA, ...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, 

In [ ]:
#Verisetimizi token halinden kaldırıp son haline getirelim
pp = pp.replace(',','', regex=True)
pp.head()

,name_with_abstract,category,tokenized,lower,no_punc,stopwords_removed,digits_removed,turkish_stem,final
0,NESNELERİ ÜÇ BOYUTLU OLARAK TARAYAN CİHAZ VE ÖN İZLEME UYGULAMASI Bu buluş mobil ve web tabanlı ...,Artırılmış ve Sanal Gerçeklik,"[NESNELERİ, ÜÇ, BOYUTLU, OLARAK, TARAYAN, CİHAZ, VE, ÖN, İZLEME, UYGULAMASI, Bu, buluş, mobil, v...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ve, ön, i̇zleme, uygulamasi, bu, buluş, mobil...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t...","[nesneleri̇, üç, boyutlu, olarak, tarayan, ci̇haz, ön, i̇zleme, uygulamasi, buluş, mobil, web, t...","[nesneleri̇, üç, boyutlu, olarak, taraya, ci̇haz, ön, i̇zle, uygulamasi, buluş, mobil, web, taba...",nesneleri̇ üç boyutlu olarak taraya ci̇haz ön i̇zle uygulamasi buluş mobil web tabanlı uygulamas...
1,BİR KÖPRÜLÜ VİNÇ SİMÜLATÖRÜNDE UYGULAMAK İÇİN BİR EĞİTİM SİSTEMİ Buluş bir operatörünün eğitilme...,Artırılmış ve Sanal Gerçeklik,"[BİR, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜNDE, UYGULAMAK, İÇİN, BİR, EĞİTİM, SİSTEMİ, Buluş, bir, operatörü...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprülü, vi̇nç, si̇mülatöründe, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir,...","[bi̇r, köprül, vi̇nç, si̇mülatör, uygulamak, i̇çi̇n, bi̇r, eği̇ti̇m, si̇stemi̇, buluş, bir, oper...",bi̇r köprül vi̇nç si̇mülatör uygulamak i̇çi̇n bi̇r eği̇ti̇m si̇stemi̇ buluş bir operatör eğitilm...
2,BİR SANAL GERÇEKLİĞİ ARTTIRILMIŞ KÖPRÜLÜ VİNÇ SİMÜLATÖRÜ Buluş bir operatörünün eğitilmesi için ...,Artırılmış ve Sanal Gerçeklik,"[BİR, SANAL, GERÇEKLİĞİ, ARTTIRILMIŞ, KÖPRÜLÜ, VİNÇ, SİMÜLATÖRÜ, Buluş, bir, operatörünün, eğiti...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttirilmiş, köprülü, vi̇nç, si̇mülatörü, buluş, bir, operatörünün, ...","[bi̇r, sanal, gerçekli̇ği̇, arttiril, köprül, vi̇nç, si̇mülatör, buluş, bir, operatör, eğitilmes...",bi̇r sanal gerçekli̇ği̇ arttiril köprül vi̇nç si̇mülatör buluş bir operatör eğitilmes bir köprül...
3,Artırılmış gerçeklik sanal gerçeklik ve robotik uygulamalara müdahale etmenizi sağlayan akıllı e...,Artırılmış ve Sanal Gerçeklik,"[Artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, ve, robotik, uygulamalara, müdahale, etmenizi, sağlaya...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a...","[artırılmış, gerçeklik, sanal, gerçeklik, robotik, uygulamalara, müdahale, etmenizi, sağlayan, a...","[artırıl, gerçeklik, sanal, gerçeklik, robotik, uygulama, müdahal, etme, sağlaya, akıllı, eldive...",artırıl gerçeklik sanal gerçeklik robotik uygulama müdahal etme sağlaya akıllı eldive platformu ...
4,GAYRİMENKULLERİ SANAL GERÇEKLİK VASITASI İLE DENEYİMLEMEYİ SAĞLAYAN WEBVR ALTYAPISINA SAHİP SANA...,Artırılmış ve Sanal Gerçeklik,"[GAYRİMENKULLERİ, SANAL, GERÇEKLİK, VASITASI, İLE, DENEYİMLEMEYİ, SAĞLAYAN, WEBVR, ALTYAPISINA, ...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan, webvr, altyapi...","[gayri̇menkulleri̇, sanal, gerçekli̇k, vasitasi, i̇le, deneyi̇mlemeyi̇, sağlayan

In [ ]:
#Verisetimizdeki her satırın legthini ölçelim. (RAM kullanımı "out of cuda memory error" yediğimiz için --> Max lenght 256 olarak sınırlamak zorundayız)
#Excel dosyasına çevrilmiş son halini incelediğimizde 157 adet verinin uzunluğunun 256 kelimeden uzun olduğunu gördük. Bu da 1560 adet olan verisetinin %10'u na tekabül ediyor.
pp['word_count'] = pp['final'].apply(lambda x: len(x.split()))
pp.to_excel('a3.xlsx', index = False, encoding="utf-8-sig")
print(pp.sort_values('word_count', ascending=False))

                                                                                       name_with_abstract  ... word_count
111   Üç boyutlu yüz modellerinden elde edilen geometri görüntüleri kullanılan derin öğrenme tabanlı ü...  ...       1542
1320  Derin sinir ağlarını kullanarak yazılım güvenlik açıklarını ve güvenlik açığına sebep olan kod d...  ...       1422
1225  Derin evrişimsel sinir ağı tabanlı kısıtsız kulak tanıma Günümüzde derin öğrenme teknolojisi ile...  ...       1411
1226  Kulak imgelerinden yaş ve cinsiyet sınıflandırma Biyometri son zamanlarda araştırmacılar tarafın...  ...       1360
341   Video analizi ile balık geçitlerinde balık tanıma ve kuyruk sallama frekansı tahmini Küresel eko...  ...       1318
...                                                                                                   ...  ...        ...
35    VR SANAL GERÇEKLİK ÜZERİNE GELİŞTİRİLEN CANLI EĞİTİM METODU Buluş VR sanal gerçeklik üzerine gel...  ...         17
1002  BİR RAPORLAMA SİST

###Pre Processing aşamasından geçen verisetimizi yeni bir dosya olarak kaydetme

In [ ]:
#Verisetimizi son aşamasına getirmek için kullandığımız ve artık ihtiyacımız olmayan kolonları kaldıralım
del pp["name_with_abstract"]
del pp["tokenized"]
del pp["lower"]
del pp["no_punc"]
del pp["stopwords_removed"]
del pp["digits_removed"]
del pp["turkish_stem"]
del pp["word_count"]

In [ ]:
#Verisetimizi yeni bir csv dosyası olarak driveımıza kaydedelim.
pp.to_csv('preprocessed_clean_dataset.csv', index = False, encoding="utf-8-sig")

In [ ]:
#Kaydettiğimiz csv dosyasını okuyalım
df = pd.read_csv('preprocessed_clean_dataset.csv')

In [ ]:
#Verisetimizin son halini kontrol edelim
df.head(10)

,category,final
0,Artırılmış ve Sanal Gerçeklik,nesneleri̇ üç boyutlu olarak taraya ci̇haz ön i̇zle uygulamasi buluş mobil web tabanlı uygulamas...
1,Artırılmış ve Sanal Gerçeklik,bi̇r köprül vi̇nç si̇mülatör uygulamak i̇çi̇n bi̇r eği̇ti̇m si̇stemi̇ buluş bir operatör eğitilm...
2,Artırılmış ve Sanal Gerçeklik,bi̇r sanal gerçekli̇ği̇ arttiril köprül vi̇nç si̇mülatör buluş bir operatör eğitilmes bir köprül...
3,Artırılmış ve Sanal Gerçeklik,artırıl gerçeklik sanal gerçeklik robotik uygulama müdahal etme sağlaya akıllı eldive platformu ...
4,Artırılmış ve Sanal Gerçeklik,gayri̇menkulleri̇ sanal gerçekli̇k vasitas i̇ deneyi̇mlemeyi̇ sağlaya webvr altyapi sahi̇p sanal...
5,Artırılmış ve Sanal Gerçeklik,çeşi̇tli̇ araç uygulanmak üzer sanal gerçekli̇k si̇stemi̇ buluş baş yolcu taş uçak uygulanmak üz...
6,Artırılmış ve Sanal Gerçeklik,bi̇r akilli ci̇haz uygulamasi buluş akıllı cihaz kullanıla uygulama ilgil olup özellik üç boyutl...
7,Artırılmış ve Sanal Gerçeklik,karma gerçekli̇k hayalet protokolü buluş deneyimci meka konumlandırma bilgi eşleştirile sanal or...
8,Artırılmış ve Sanal Gerçeklik,dok kolaylaştırma siste dok süreç dok kadı görme işitme kok alma duyu etki edip zihin olumlu oda...
9,Artırılmış ve Sanal Gerçeklik,sanal gerçekli̇k ortam oy terapi̇ si̇stemi̇ buluş dikkat konsantrasyo hafız denge-koordinasyo fo...


###Verisetini kategorilerine göre etiketleme

In [ ]:
possible_labels = df.category.unique()

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
#Verinin kategoriye dair etiketini alır
df['label'] = df.category.replace(label_dict)

In [ ]:
#İlk 10 veriyi etiketleri ve text içeriği ile birlikte görelim
df.head(600)

,category,final,label
0,Artırılmış ve Sanal Gerçeklik,nesneleri̇ üç boyutlu olarak taraya ci̇haz ön i̇zle uygulamasi buluş mobil web tabanlı uygulamas...,0
1,Artırılmış ve Sanal Gerçeklik,bi̇r köprül vi̇nç si̇mülatör uygulamak i̇çi̇n bi̇r eği̇ti̇m si̇stemi̇ buluş bir operatör eğitilm...,0
2,Artırılmış ve Sanal Gerçeklik,bi̇r sanal gerçekli̇ği̇ arttiril köprül vi̇nç si̇mülatör buluş bir operatör eğitilmes bir köprül...,0
3,Artırılmış ve Sanal Gerçeklik,artırıl gerçeklik sanal gerçeklik robotik uygulama müdahal etme sağlaya akıllı eldive platformu ...,0
4,Artırılmış ve Sanal Gerçeklik,gayri̇menkulleri̇ sanal gerçekli̇k vasitas i̇ deneyi̇mlemeyi̇ sağlaya webvr altyapi sahi̇p sanal...,0
...,...,...,...
595,Haberleşme ve Sinyalleşme,myostat inhibitör kullan kombinasyo tedavi mevcut tarifna myostat sinyalleşme inhip e bir aja ku...,2
596,Haberleşme ve Sinyalleşme,ayrıl çağrı kontrolü konumlandırıç kontrolü bir iletiş şebeke esneklik geliştirme yönte ayrıl ça...,2
597,Haberleşme ve Sinyalleşme,bir an kart devres mevcut buluş geliştirile televizyo an kart devres a devre kart b işlemci geçi...,2
598,Haberleşme ve Sinyalleşme,si̇nyal kariştiriç ci̇haz konumlari tespi̇t edi̇lmesi̇ni̇ sağlaya bi̇r si̇ste buluş özel alan po...,2


In [ ]:
#Verisetimizin Türkçe stem olmuş hali ile en çok kullanılan kelimelerini bulalım
df2 = (df['final'].str.split(expand=True)
              .stack()
              .value_counts()
              .rename_axis('vals')
              .reset_index(name='count'))
print(df2)


               vals  count
0               bir   7469
1            olarak   2602
2               gör   2235
3               ver   2222
4             siste   1979
...             ...    ...
17931  audio-visual      1
17932     i̇şlemi̇n      1
17933    alternatör      1
17934  rastlanılmak      1
17935            ni      1

[17936 rows x 2 columns]


## Eğitim ve Doğrulama Setlerinin Ayrılması

In [ ]:
#Veriyi eğitim, test, doğrulama olarak ayırmak için gereken kütüphaneleri yükleyelim
from sklearn.model_selection import train_test_split

In [ ]:
#Veriyi eğitim ve test için eğitim ve doğrulama olarak ayırır, rastgelelik, test büyüklüğü belirlenir.
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

Veriyi ayırdıktan sonra eğitim kümesinde mi doğrulama kümesinde mi görebiliriz. Daha sonra hangi kategoriler var bu kategorilere atanan etiketler ve hangi kümede kaç veri olduğunun istatistiklerini tablo olarak çıkarabiliriz.

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['category', 'label', 'data_type']).count()

final
category                      label data_type       
Artırılmış ve Sanal Gerçeklik 0     train         99
                                    val           25
Görüntü İşleme                1     train        350
                                    val           88
Haberleşme ve Sinyalleşme     2     train        174
                                    val           43
IoT                           3     train        147
                                    val           36
Yapay Zeka                    4     train        478
                                    val          120

## Tokenizer Yükleme ve Datayı Kodlama

Gerekli kurulumlarla başlayalım.

In [ ]:
!pip3 install transformers

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
#BERT Tokenizerın pretrained olarak alınması
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', 
                                          do_lower_case=True)

In [ ]:
#Verilerin eğitim ve doğrulama olarak encode edilmesi
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].final.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True,
    truncation=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].final.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    truncation=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
#Eğitim setinin uzunluğu
len(dataset_train)

1248

In [ ]:
#Doğrulama setinin uzunluğu
len(dataset_val)

312

## Önceden Eğitilen BERT Modelini Kullanma

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-128k-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)



Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassificatio

## Veri Yükleyicileri (DataLoaders) Oluşturma

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

## Optimize Edici ve Zamanlayıcıyı(Scheduler) Kurma 

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=5e-5,
                  eps=1e-8)

In [ ]:
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Performans için F1 Score'un Kullanılması

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
#F1 Score hesaplaması için fonksiyon
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
#Her bir sınıf için doğruluk hesabı fonksiyonu
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Eğitim için döngünün oluşturulması


In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

total_t0 = time.time()

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    print("Training completed in {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



Epoch 1
Training loss: 1.1476024091243744
Validation loss: 0.46723655313253404
F1 Score (Weighted): 0.8679586086034059
Training completed in 0:01:10 (h:mm:ss)



Epoch 2
Training loss: 0.27509423784720594
Validation loss: 0.2051113910973072
F1 Score (Weighted): 0.9327963014284345
Training completed in 0:02:19 (h:mm:ss)



Epoch 3
Training loss: 0.14886259913253477
Validation loss: 0.18108112998306752
F1 Score (Weighted): 0.9486263314064144
Training completed in 0:03:28 (h:mm:ss)



Epoch 4
Training loss: 0.09698174165514035
Validation loss: 0.16877338252961635
F1 Score (Weighted): 0.9551028847435388
Training completed in 0:04:37 (h:mm:ss)



In [ ]:
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-128k-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassificatio

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/BertTest/finetuned_BERT_epoch_4.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: Artırılmış ve Sanal Gerçeklik
Accuracy: 24/25

Class: Görüntü İşleme
Accuracy: 82/88

Class: Haberleşme ve Sinyalleşme
Accuracy: 42/43

Class: IoT
Accuracy: 35/36

Class: Yapay Zeka
Accuracy: 115/120



In [ ]:
print(predictions)

In [ ]:
print(evaluate(dataloader_validation))